# In Notebook 2 is an incomplete process of refining the features collected into a format to model with the panel dataframe. While I did clean a decent portion of the data scraped there are still more to refine, and of the ones we did, there is still the challenge of missing values. In this notebook we'll handle the NaNs. 

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
import os

In [30]:
#to start like in 2, let's just manage one 
#dataset at a time
soybean_yield = pd.read_csv('../dat/SOYBEANS-AcreageYieldProductionandPrice-2019-11-21.csv')
fertilizers = pd.read_csv('../dat/fertilizers_cleaned.csv')

In [31]:
soybean_yield

,YEAR,LOCATION,STATE ANSI,ASD CODE,COUNTY ANSI,REFERENCE PERIOD,COMMODITY,AREA HARVESTED in ACRES,AREA PLANTED in ACRES,PRODUCTION in BU,YIELD in BU / ACRE,PRODUCTION in $,PRICE RECEIVED in $ / BU
0,1996,ALABAMA,01,,,YEAR,SOYBEANS,"305,000","320,000","10,370,000",34,"76,738,000",
1,1996,ALABAMA,01,,,MARKETING YEAR,SOYBEANS,,,,,,7.4
2,1996,ARKANSAS,05,,,YEAR,SOYBEANS,"3,500,000","3,550,000","112,000,000",32,"824,320,000",
3,1996,ARKANSAS,05,,,MARKETING YEAR,SOYBEANS,,,,,,7.36
4,1996,DELAWARE,10,,,YEAR,SOYBEANS,"217,000","220,000","7,595,000",35,"54,684,000",
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1443,2019,SOUTH DAKOTA,46,,,YEAR,SOYBEANS,"3,560,000","3,600,000","153,080,000",43,,
1444,2019,TENNESSEE,47,,,YEAR,SOYBEANS,"1,380,000","1,400,000","64,860,000",47,,
1445,2019,TEXAS,48,,,YEAR,SOYBEANS,"68,000","80,000","2,108,000",31,,
1446,2019,VIRGINIA,51,,,YEAR,SOYBEANS,"560,000","570,000","19,040,000",34,,


In [32]:
def make_numbers(cleaned_df_):
    cleaned_df = cleaned_df_.copy(deep=True)
    for col in cleaned_df.columns[7:]: #5 for soybean yield
        cleaned_df.fillna('-1',inplace=True)
        cleaned_df[col] = cleaned_df[col].apply(lambda z:
                       z.replace('(D)','0'))
        cleaned_df[col] = cleaned_df[col].apply(lambda z:
                       z.replace('(NA)','-1'))
        cleaned_df[col] = cleaned_df[col].apply(lambda z:
                       z.replace('(Z)','0'))

        cleaned_df[col] = cleaned_df[col].apply(lambda z:
                       z.replace(' ','0'))
        cleaned_df[col] = cleaned_df[col].apply(lambda z:
                       z.replace(',',''))
        cleaned_df[col] = cleaned_df[col].apply(lambda z:
                                               float(z))    
    
    return cleaned_df.replace(-1,np.nan)

In [33]:
soybean_yield = make_numbers(soybean_yield)

In [34]:
fertilizers.YEAR.nunique()

13

In [35]:
fertilizers.LOCATION.nunique()

22

In [36]:
soybean_yield.YEAR.nunique()

24

In [37]:
soybean_yield.LOCATION.nunique()

32

So what do we want to do? See what states and/or years they share. 

In [38]:
for loc in soybean_yield.LOCATION.unique():
    if loc not in fertilizers.LOCATION.unique():
        print(loc , 'will get tossed. ')

ALABAMA will get tossed. 
FLORIDA will get tossed. 
GEORGIA will get tossed. 
NEW JERSEY will get tossed. 
OKLAHOMA will get tossed. 
SOUTH CAROLINA will get tossed. 
TEXAS will get tossed. 
NEW YORK will get tossed. 
WEST VIRGINIA will get tossed. 
OTHER STATES will get tossed. 


These are states we have yield data for but no fertilizer data for. 

In [42]:
yield_fertilizer = soybean_yield.merge(fertilizers,on=['YEAR','LOCATION'],how='left')

In [43]:
yield_fertilizer

,YEAR,LOCATION,STATE ANSI,ASD CODE,COUNTY ANSI,REFERENCE PERIOD,COMMODITY,AREA HARVESTED in ACRES,AREA PLANTED in ACRES,PRODUCTION in BU,...,"FERTILIZER: (PHOSPHATE)APPLICATIONS in LB / ACRE / YEAR, AVG","FERTILIZER: (PHOSPHATE)APPLICATIONS in NUMBER, AVG","FERTILIZER: (POTASH)APPLICATIONS in LB / ACRE / APPLICATION, AVG","FERTILIZER: (POTASH)APPLICATIONS in LB / ACRE / YEAR, AVG","FERTILIZER: (POTASH)APPLICATIONS in NUMBER, AVG","FERTILIZER: (POTASH)TREATED in PCT OF AREA PLANTED, AVG","FERTILIZER: (SULFUR)APPLICATIONS in LB / ACRE / APPLICATION, AVG","FERTILIZER: (SULFUR)APPLICATIONS in LB / ACRE / YEAR, AVG","FERTILIZER: (SULFUR)APPLICATIONS in NUMBER, AVG","FERTILIZER: (SULFUR)TREATED in PCT OF AREA PLANTED, AVG"
0,1996,ALABAMA,01,,,YEAR,SOYBEANS,305000.0,320000.0,10370000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1996,ALABAMA,01,,,MARKETING YEAR,SOYBEANS,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1996,ARKANSAS,05,,,YEAR,SOYBEANS,3500000.0,3550000.0,112000000.0,...,48.0,1.0,60.0,60.0,1.0,43.0,NaN,NaN,NaN,NaN
3,1996,ARKANSAS,05,,,MARKETING YEAR,SOYBEANS,0.0,0.0,0.0,...,48.0,1.0,60.0,60.0,1.0,43.0,NaN,NaN,NaN,NaN
4,1996,DELAWARE,10,,,YEAR,SOYBEANS,217000.0,220000.0,7595000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1443,2019,SOUTH DAKOTA,46,,,YEAR,SOYBEANS,3560000.0,3600000.0,153080000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1444,2019,TENNESSEE,47,,,YEAR,SOYBEANS,1380000.0,1400000.0,64860000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1445,2019,TEXAS,48,,,YEAR,SOYBEANS,68000.0,80000.0,2108000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1446,2019,VIRGINIA,51,,,YEAR,SOYBEANS,560000.0,570000.0,19040000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# And repeat for other features. But prior to doing so, the question is: What should we do with NaNs?

In [49]:

for group, g in soybean_yield.groupby(['LOCATION']):
    print(g['LOCATION'].values[0])
    g.sort_values('YEAR',ascending=True)
    g = g.loc[g['REFERENCE PERIOD'] == 'YEAR']
    print(g['PRODUCTION in BU'].autocorr())

ALABAMA
0.7763748936132449
ARKANSAS
0.8170725350228107
DELAWARE
-0.01938482289423457
FLORIDA
0.6366755957929003
GEORGIA
0.5243815663677817
ILLINOIS
0.6188321873503511
INDIANA
0.4895352946567495
IOWA
0.35168639926965717
KANSAS
0.8363752941873849
KENTUCKY
0.8118362565219838
LOUISIANA
0.7965586167566318
MARYLAND
0.4458337267217517
MICHIGAN
0.4731084999748729
MINNESOTA
0.6107287472088719
MISSISSIPPI
0.8547160289495961
MISSOURI
0.4933933653128249
NEBRASKA
0.8544058637435356
NEW JERSEY
0.168835944857834
NEW YORK
0.8420328103088448
NORTH CAROLINA
0.3831255689606167
NORTH DAKOTA
0.9158545566876942
OHIO
0.6046798793126069
OKLAHOMA
0.6555458783619641
OTHER STATES
nan
PENNSYLVANIA
0.8795380242617414
SOUTH CAROLINA
-0.1197535637583689
SOUTH DAKOTA
0.8109789033780239
TENNESSEE
0.7461228025796723
TEXAS
0.400294860442016
VIRGINIA
0.5985222078548583
WEST VIRGINIA
0.8573139368652077
WISCONSIN
0.7964938519910774


In [48]:
g = g.loc[g['REFERENCE PERIOD'] == 'YEAR']